In [1]:
import pandas as pd
from rdkit import Chem

def replace_first_space(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            first_space_index = line.find(' ')
            modified_line = line[:first_space_index] + ';' + line[first_space_index + 1:]
            outfile.write(modified_line)

In [2]:
replace_first_space("./original_outputs/bad0.smi", "./parsed_outputs/parsed_smi_0.smi")
replace_first_space("./original_outputs/bad1.smi", "./parsed_outputs/parsed_smi_1.smi")
replace_first_space("./original_outputs/bad2.smi", "./parsed_outputs/parsed_smi_2.smi")
replace_first_space("./original_outputs/bad3.smi", "./parsed_outputs/parsed_smi_3.smi")

In [3]:
df0 = pd.read_csv("./parsed_outputs/parsed_smi_0.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df1 = pd.read_csv("./parsed_outputs/parsed_smi_1.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df2 = pd.read_csv("./parsed_outputs/parsed_smi_2.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df3 = pd.read_csv("./parsed_outputs/parsed_smi_3.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df = pd.concat([df0, df1, df2, df3])
df

,SMILES,FlagReason
0,C#CC1=CC=C(OCC2=N[N]C=C2C2=CC=CC3=C2NC(=C3CCCO...,TP1 abnormal_valence
1,C1=CC(=S)SS1,TP1 no_interesting_atoms
2,C1=CSC(=S)S1,TP1 no_interesting_atoms
3,C1C2=C(C(=S)SS2)CC1,TP1 no_interesting_atoms
4,OB(O)C1=CC(=C(C2=CNC3=NC=C(C4=CN=CC(=C4)C(=O)N...,TP1 non_allowed_atom
...,...,...
21,C1=NN2C=CC(=NC2=C1C(=O)NC1=CN(N=C1C(F)F)C1CCC(...,: D(130) too_many_rings:acetylene
22,OC(=O)[C@@H](CC1=CC=CC(=C1)CN(CC1=CC=CC(=C1)C[...,: D(250) positive:negative
23,OC[C@H]1O[C@@H](O[C@@H]2OC=C3C(=O)OCC[C@@]3(O)...,: D(110) acetal_1_in_ring:michael_demerited
24,C1=CC(=CC2=C1OCCO2)C1=NN(C(=O)C)[C@@H](C2=CC=C...,: D(100) nitro:acyl_hydrazone_cyclic


In [4]:
df_fail = df[~df.FlagReason.str.contains("match")]
df_match = df[df.FlagReason.str.contains("match")]
df_match["InChI"] = df_match.SMILES.apply(lambda x: Chem.MolToInchi(Chem.MolFromSmiles(x)))
df_match["eli_match"] = 1

[22:14:54] WARNING: Charges were rearranged

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Charges were rearranged

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefined stereo

[22:14:54] WARNING: Omitted undefine

In [5]:
test_data_df = pd.read_csv("../data/SMILES_test/test_data_all.csv")

In [13]:
df_final = test_data_df.merge(df_match, how="left", on=["InChI"])[["InChI", "covalent", "FlagReason", "eli_match"]]
df_final.eli_match = df_final.eli_match.fillna(0)
df_final

,InChI,covalent,FlagReason,eli_match
0,InChI=1S/C25H26N8O3/c1-2-26-24(36)20-13-22(28-...,1,(1 matches to 'acetylene_heteroatom'),1.0
1,InChI=1S/C25H26N8O3/c1-2-26-24(36)20-13-22(28-...,1,(1 matches to 'acetylene_heteroatom'),1.0
2,InChI=1S/C23H20N4O4/c1-27(2)23(30)15-4-13(8-24...,1,(1 matches to 'aldehyde'),1.0
3,InChI=1S/C13H17NO4S/c1-10-7-14(8-11(2)18-10)19...,1,(1 matches to 'aldehyde'),1.0
4,"InChI=1S/C17H10F3N3O2/c18-17(19,20)11-5-7-14(2...",1,(1 matches to 'aldehyde'),1.0
...,...,...,...,...
606,"InChI=1S/C5H9N/c1-5-3-2-4-6-5/h2-4H2,1H3",0,NaN,0.0
607,InChI=1S/C10H18O2/c1-8(2)5-4-6-10(3)9(7-11)12-...,0,(1 matches to 'het_3mem_ring'),1.0
608,"InChI=1S/C10H18O2/c1-8(6-7-11)4-5-9-10(2,3)12-...",0,(1 matches to 'het_3mem_ring'),1.0
609,"InChI=1S/C10H20O2/c1-8(6-7-11)4-5-9-10(2,3)12-...",0,(1 matches to 'het_3mem_ring'),1.0


In [16]:
from sklearn.metrics import precision_score, recall_score
(
    precision_score(df_final.covalent, df_final.eli_match),
    recall_score(df_final.covalent, df_final.eli_match),
)


(0.8711484593837535, 0.731764705882353)